In [ ]:
# --- Installation & Setup (Colab only) ---
# Clone our GitHub repository into the Colab environment

import sys

# Check if the env run under Colab
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !git clone https://github.com/weizmannk/EarthOrbitPlan.git
    %cd EarthOrbitPlan
    !pip install -e .

    print("Environment ready. You can now run the rest of the notebook.")

In [ ]:
import os
import sys

# Check if the env run under Colab
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    # Go to the "EarthOrbitPlan/earthorbitplan/tutorials"
    os.chdir("./earthorbitplan/tutorials")

    # Check if the 'kilonovae_detection_rate.ipynb' is there
    print(os.listdir())

In [ ]:
# Suppress known warnings for cleaner output
import warnings

warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
warnings.filterwarnings("ignore", ".*dubious year.*")
warnings.filterwarnings(
    "ignore", "Tried to get polar motions for times after IERS data is valid.*"
)

In [ ]:
import logging

import numpy as np
from astropy import units as u
from astropy.table import QTable
from IPython.display import Math, display
from scipy import stats

from earthorbitplan.probability.rate import poisson_lognormal_rate_quantiles
from earthorbitplan.utils.path import get_project_root

### Setup logging 

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    force=True,
)

In [ ]:
import os

print("cwd =", os.getcwd())
print(
    "events.ecsv exists?", os.path.exists("../../data/events_ultrasat_non_overlap.ecsv")
)

### Load simulated event data

In [ ]:
root = get_project_root()
events_file = root / "data" / "events_ultrasat_non_overlap.ecsv"
main_table = QTable.read(events_file)

events_file

### Get unique run names

In [ ]:
runs = np.unique(main_table["run"])

print(runs)

### Filter events by objective_value cutoff

In [ ]:
cutoff = main_table["cutoff"][0]
main_table = main_table[main_table["objective_value"] >= cutoff]
main_table[0:2]

### Group events by run

In [ ]:
event_tables_by_run = {run: main_table[main_table["run"] == run] for run in runs}
event_tables_by_run[runs[0]][0:2]

### Set merger rate priors from O3 R&P Table II (last column)

This section explains how we reproduce the 5%, 50%, and 95% quantiles of the BNS and NSBH merger rate, 

as reported in [O3 R&P, Table II](https://doi.org/10.1103/PhysRevX.13.011048), for our own simulation statistics.


### Reported CBC merger rates:

- **5% quantile:** 100 $\mathrm{Gpc}^{-3}\ \mathrm{yr}^{-1} $

- **50% quantile (median):** 240 $\mathrm{Gpc}^{-3}\ \mathrm{yr}^{-1} $

- **95% quantile:** 510 $\mathrm{Gpc}^{-3}\ \mathrm{yr}^{-1} $

In [ ]:
lo, mid, hi = 100, 240, 510  # In Gpc^-3 yr^-1

# 90% interval width (in log-normal)
(standard_90pct_interval,) = np.diff(stats.norm.interval(0.9))
log_target_rate_mu = np.log(mid)
log_target_rate_sigma = np.log(hi / lo) / standard_90pct_interval

print(log_target_rate_mu, log_target_rate_sigma)

### Compute effective rate for each run:

In [ ]:
# Get effective rate for each run

log_simulation_effective_rate_by_run = {
    key: np.log(value.to_value(u.Gpc**-3 * u.yr**-1))
    for key, value in main_table.meta["effective_rate"].items()
}
log_simulation_effective_rate_by_run

### Compute median ($\mu$) and quantiles for each run  : $\mu$ is calculated for each simulation run as:

$
\mu = \log(\text{target median}) + \log(\text{run duration}) - \log(\text{simulation effective rate}) + \log(\text{N or detected N})
$

Here we use both number of events and number detected (with known position).


In [ ]:
prob_quantiles = np.asarray([0.5, 0.05, 0.95])  # median, 5%, 95%
run_duration = 0.5  # years

mu = np.asarray(
    [
        log_target_rate_mu
        + np.log(run_duration)
        - log_simulation_effective_rate_by_run[run]
        + np.log(
            [
                np.sum(_)
                for _ in [
                    np.ones_like(event_tables_by_run[run]["objective_value"]),
                    event_tables_by_run[run]["detection_probability_known_position"],
                ]
            ]
        )
        for run in runs
    ]
)


labels = ["All selected", "Detected known position"]
display(Math(r"\mu\ \mathrm{values\ per\ run:}"))

for run, vals in zip(runs, mu):
    for label, v in zip(labels, vals):
        safe_label = label.replace(" ", "~")
        display(Math(rf"{run}~\mathrm{{({safe_label})}}:~{v:.3f}"))

### Compute Poisson-Lognormal rate quantiles for all runs

This step calculates the quantile intervals for the merger rates for all runs.


In [ ]:
rate_quantiles = poisson_lognormal_rate_quantiles(
    prob_quantiles[np.newaxis, np.newaxis, :],
    mu.T[:, :, np.newaxis],
    log_target_rate_sigma,
)


quantile_labels = ["Median", "5%", "95%"]
idx = 0

print(f"Quantiles for the run {runs[idx]}:")
# Print column headers
print("  {:<25} {:>8} {:>8} {:>8}".format("Type", *quantile_labels))
for label, row in zip(labels, rate_quantiles[idx]):
    print("  {:<25} {:>8.2f} {:>8.2f} {:>8.2f}".format(label, *row))

### Detection  and Selected Rate

In [ ]:
def make_rst_table(headers, rows):
    columns = [headers] + rows
    n_cols = len(headers)
    col_widths = [max(len(str(row[i])) for row in columns) for i in range(n_cols)]

    def sep(char="+", fill="-"):
        return char + char.join(fill * (w + 2) for w in col_widths) + char

    def fmt_row(row):
        return (
            "| "
            + " | ".join(str(cell).ljust(w) for cell, w in zip(row, col_widths))
            + " |"
        )

    lines = [
        sep(),
        fmt_row(headers),
        sep("=", "="),
    ]
    for row in rows:
        lines.append(fmt_row(row))
        lines.append(sep())
    return "\n".join(lines)


# Prepare headers and format quantile results
headers = ["Run"] + list(runs)
labels = ["Number of events selected", "Number of events detected"]
rst_rows = []

for label, row in zip(labels, rate_quantiles):
    formatted = [
        "${}_{{-{}}}^{{+{}}}$".format(*np.rint([mid, mid - lo, hi - mid]).astype(int))
        for mid, lo, hi in row
    ]
    rst_rows.append([label] + formatted)

rst_table = make_rst_table(headers, rst_rows)

# Print the table
print(rst_table)

In [ ]:
%matplotlib inline
from earthorbitplan.utils.path import get_project_root
from earthorbitplan.workflow.area_distance import plot_area_distance

root = get_project_root()
events_file = root / "data" / "events_ultrasat_non_overlap.ecsv"
plot_area_distance(events_file, show=True)